## Match Index Notebook

This Notebook is for index functions

In [30]:
import pandas as pd
import os
import re

In [31]:
parent_dir = os.path.abspath('../..') # ".." means go back 1 directory!
print(parent_dir)

c:\Users\icesk\OneDrive\Desktop\tennis_consulting\consulting-spring2025


In [32]:
file_path = os.path.join(parent_dir, 'data/mens/mens_results.csv')
file_path


'c:\\Users\\icesk\\OneDrive\\Desktop\\tennis_consulting\\consulting-spring2025\\data/mens/mens_results.csv'

In [33]:
matches = pd.read_csv(file_path)
matches.head()

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score
0,"Dual Match: University of California, Los Ange...",2025-04-26,Emon Van Loben Sels,13.59,Alexander Bernard,13.33,"7-6(5), 5-7, 6-3"
1,"Dual Match: University of California, Los Ange...",2025-04-26,Alexander Hoogmartens,13.00,Jack Anthrop,13.66,"7-5, 0-6, 6-4"
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2"
3,"Dual Match: University of California, Los Ange...",2025-04-26,Preston Stearns,13.08,Aadarsh Tripathi,12.94,"6-3, 2-6, 6-2"
4,"Dual Match: University of California, Los Ange...",2025-04-26,Kaylan Bigun,13.17,William Jansen,13.24,"6-2, 7-5"


In [34]:
# Set UCLA name
my_school = "University of California, Los Angeles"

# Extract opponent name from 'Event Name'
def get_opponent(event):
    if not isinstance(event, str):
        return None
    teams = re.findall(r'vs\s(.+)', event)
    if my_school in event and teams:
        return teams[0].strip() if my_school in event.split('vs')[0] else event.split('vs')[0].replace('Dual Match:', '').strip()
    return None

matches['opponent_school'] = matches['Event Name'].apply(get_opponent)

# Set of UCLA players
ucla_players = {
    "Andrei Crabel", "Alexander Hoogmartens", "Spencer Johnson", "Rudy Quan",
    "Giacomo Revelli", "Aadarsh Tripathi", "Gianluca Ballotta", "Kaylan Bigun",
    "Cassius Chinlund", "Emon van Loben Sels", "Leo von Bismarck"
}

matches['opponent'] = matches.apply(
    lambda row: row['Player2'] if row['Player1'] in ucla_players else (
        row['Player1'] if row['Player2'] in ucla_players else None
    ),
    axis=1
)

matches['ucla_result'] = matches.apply(
    lambda row: 'Win' if row['Player1'] in ucla_players else (
        'Loss' if row['Player2'] in ucla_players else None
    ),
    axis=1
)

# Helper function to flip scores like "7-6(5), 5-7, 6-3"
def flip_score(score_str):
    if not isinstance(score_str, str):
        return score_str
    flipped = []
    for set_score in score_str.split(', '):
        match = re.match(r'(\d+)-(\d+)(\(\d+\))?', set_score)
        if match:
            p1, p2, tiebreak = match.groups()
            flipped_score = f"{p2}-{p1}"
            if tiebreak:
                flipped_score += tiebreak
            flipped.append(flipped_score)
        else:
            flipped.append(set_score)  # Preserve unrecognized formats
    return ', '.join(flipped)

# Flip the score only if Player 2 is a UCLA player
matches['Score (UCLA Perspective)'] = matches.apply(
    lambda row: flip_score(row['Score']) if row['Player2'] in ucla_players else row['Score'],
    axis=1
)

matches

# filter through each player on ucla team
## Gianluca Ballotta
gianlucaBallotta_df = matches[(matches['Player1'] == 'Gianluca Ballotta') | (matches['Player2'] == 'Gianluca Ballotta')]
gianlucaBallotta_df
## Kaylan Bigun
kaylanBigun_df = matches[(matches['Player1'] == 'Kaylan Bigun') | (matches['Player2'] == 'Kaylan Bigun')]
kaylanBigun_df
## Cassius Chinlund
cassius_df = matches[(matches['Player1'] == 'Kaylan Bigun') | (matches['Player2'] == 'Kaylan Bigun')]
kaylanBigun_df
## Andrei Crabel
## Alexander Hoogmartens
## Spencer Johnson
## Rudy Quan
rudy_df = matches[(matches['Player1'] == 'Rudy Quan') | (matches['Player2'] == 'Rudy Quan')]
rudy_df
## Giacomo Revelli
## Aadarsh Tripathi
## Emon van Loben Sels
## Leo von Bismarck

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score,opponent_school,opponent,ucla_result,Score (UCLA Perspective)
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2",The Ohio State University,Aidan Kim,Loss,"6-7(3), 2-6"
10,Dual Match: Michigan State University vs Unive...,2025-04-25,Rudy Quan,13.67,Aristotelis THANOS,13.61,"4-6, 3-1",Michigan State University,Aristotelis THANOS,Win,"4-6, 3-1"
17,Dual Match: University of Michigan vs Universi...,2025-04-24,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 5-6",University of Michigan,William Cooksey,Win,"6-3, 5-6"
23,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,13.67,Calvin MUELLER,13.00,"3-6, 6-3, 6-1",University of Nebraska-Lincoln,Calvin MUELLER,Win,"3-6, 6-3, 6-1"
25,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,12.00,Rudy Quan,13.67,"2-6, 6-3",University of Wisconsin-Madison,Michael Minasyan,Loss,"6-2, 3-6"
...,...,...,...,...,...,...,...,...,...,...,...
4790,NaN,2014-05-10,Rudy Quan,13.67,Matthew Trinidad,2.00,"6-0, 6-0",None,Matthew Trinidad,Win,"6-0, 6-0"
4804,NaN,2014-04-12,Kyle Kang,13.00,Rudy Quan,13.67,"6-0, 6-1",None,Kyle Kang,Loss,"0-6, 1-6"
4805,NaN,2014-04-12,Rudy Quan,13.67,Prayag Ahire,9.00,"6-2, 6-0",None,Prayag Ahire,Win,"6-2, 6-0"
4810,NaN,2014-04-11,Rudy Quan,13.67,Heaton Manor,0.00,"6-0, 6-0",None,Heaton Manor,Win,"6-0, 6-0"


In [36]:
final_df = rudy_df[['opponent_school', 'opponent', 'Score (UCLA Perspective)', 'Date']].rename(columns={
    'Score (UCLA Perspective)': 'score',
    'Date': 'date'
})

final_df

,opponent_school,opponent,score,date
2,The Ohio State University,Aidan Kim,"6-7(3), 2-6",2025-04-26
10,Michigan State University,Aristotelis THANOS,"4-6, 3-1",2025-04-25
17,University of Michigan,William Cooksey,"6-3, 5-6",2025-04-24
23,University of Nebraska-Lincoln,Calvin MUELLER,"3-6, 6-3, 6-1",2025-04-19
25,University of Wisconsin-Madison,Michael Minasyan,"6-2, 3-6",2025-04-17
...,...,...,...,...
4790,None,Matthew Trinidad,"6-0, 6-0",2014-05-10
4804,None,Kyle Kang,"0-6, 1-6",2014-04-12
4805,None,Prayag Ahire,"6-2, 6-0",2014-04-12
4810,None,Heaton Manor,"6-0, 6-0",2014-04-11


In [ ]:
# Create a function that returns data in this format
file_path = os.path.join(parent_dir, 'data/index_sample.csv')
sample_output = pd.read_csv(file_path)
sample_output

In [ ]:
# Column Format Guidelines:

# school_name: Full School Name
# opponent: Full Name
# score: Respect to Player we are querying for (switch score if ucla player lost)
# date: 00/00/00